In [94]:
import sys
sys.path.append("/Users/india.kerlenesta/Projects/ojd_daps_extension/ojd_daps_skills")

In [95]:
"""Evaluation analysis - 

1) We compare lightcast's algorithm with our lightcast mapped skills at the skill level. We guarantee to map to skills based on setting the minimum cosine similarity threshold to 0;
2) We also compare top extracted skills per occupation with ESCO's essential skills.
"""

"Evaluation analysis - \n\n1) We compare lightcast's algorithm with our lightcast mapped skills at the skill level. We guarantee to map to skills based on setting the minimum cosine similarity threshold to 0;\n2) We also compare top extracted skills per occupation with ESCO's essential skills.\n"

In [96]:
from ojd_daps_skills import config, bucket_name, logger
from ojd_daps_skills.getters.data_getters import (
    get_s3_resource,
    load_s3_data,
    save_to_s3,
)
import pandas as pd
import random
import ast

### 0. Relevant functions for analysis.

In [97]:
def percent_overlap(ojo_skills, lightcast_skills):
    """Calculate the percent overlap between two lists"""
    
    if len(ojo_skills) and len(lightcast_skills) > 0:

        setA = set(ojo_skills)
        setB = set(lightcast_skills)

        overlap = setA & setB
        universe = setA | setB


        result1 = float(len(overlap)) / len(setA) * 100
        result2 = float(len(overlap)) / len(setB) * 100
        result3 = float(len(overlap)) / len(universe) * 100

        return result1, result2, result3
    
    else:
        return 100, 100, 100

### 1. Relevant parameters and datasets for analysis.

In [98]:
s3 = get_s3_resource()
ojo_lightcast_skills = load_s3_data(s3, bucket_name, 'escoe_extension/outputs/evaluation/ojo_esmi_skills/ojo_lightcast_skills_20221115.json')
esco_occupations = load_s3_data(s3, bucket_name, 'escoe_extension/outputs/evaluation/aggregate_ojo_esco/ojo_esco_occupation_skills_results_v2.json')
esco_occupations_df = pd.DataFrame(esco_occupations).T.sort_values('skills_in_ojo_esco_percent', ascending=False).reset_index().rename(columns={'index':'occupation'})
esco_skills = load_s3_data(s3, bucket_name, 'escoe_extension/outputs/data/skill_ner_mapping/esco_data_formatted.csv')
esco_hier_mapper = load_s3_data(s3, bucket_name, 'escoe_extension/outputs/data/skill_ner_mapping/esco_hier_mapper.json')

### 2. Lightcast comparison analysis

In [6]:
for job_id, skill_info in ojo_lightcast_skills.items():
    comps = percent_overlap(skill_info['ojo_skills'], skill_info['lightcast_skills'])
    for comp_type, comp in zip(('ojo_skills_overlap', 'lightcast_skills_overlap', 'universal_overlap'), comps):
        skill_info[comp_type] = comp    

In [7]:
ojo_lightcast_skills_df = pd.DataFrame(ojo_lightcast_skills).T
ojo_lightcast_skills_df = ojo_lightcast_skills_df.sort_values('lightcast_skills_overlap', ascending=False)
ojo_lightcast_skills_df = ojo_lightcast_skills_df[~(ojo_lightcast_skills_df['ojo_skills'].str.len() == 0) & (ojo_lightcast_skills_df['lightcast_skills'].str.len() != 0)]

In [8]:
print('percent overlap analysis')

print(f"the % of job adverts with no skills overlap is: {len(ojo_lightcast_skills_df[ojo_lightcast_skills_df['ojo_skills_overlap'] == 0.0])/len(ojo_lightcast_skills_df)}")
print(f"the average # of lightcast skills we extract is: {ojo_lightcast_skills_df.ojo_skills.apply(lambda x: len(x)).mean()}")
print(f"the median # of lightcast skills we extract is: {ojo_lightcast_skills_df.ojo_skills.apply(lambda x: len(x)).median()}")

print(f"the average # of lightcast skills lightcast extracts is: {ojo_lightcast_skills_df.lightcast_skills.apply(lambda x: len(x)).mean()}")
print(f"the median # of lightcast skills lightcast extracts is: {ojo_lightcast_skills_df.lightcast_skills.apply(lambda x: len(x)).median()}")

print(f"of the job adverts with overlap, on average, {ojo_lightcast_skills_df[ojo_lightcast_skills_df['lightcast_skills_overlap'] != 0.0].lightcast_skills_overlap.mean()} of lightcast skills are present in our current approach.")
print(f"of the job adverts with overlap, the median is {ojo_lightcast_skills_df[ojo_lightcast_skills_df['lightcast_skills_overlap'] != 0.0].lightcast_skills_overlap.median()} of lightcast skills are present in our current approach.")

print(f"of the job adverts with overlap, on average, {ojo_lightcast_skills_df[ojo_lightcast_skills_df['ojo_skills_overlap'] != 0.0].ojo_skills_overlap.mean()} of our skills are present in lighcast skills.")
print(f"of the job adverts with overlap, the median is {ojo_lightcast_skills_df[ojo_lightcast_skills_df['ojo_skills_overlap'] != 0.0].ojo_skills_overlap.median()} of our skills are present in lightcast skills.")

percent overlap analysis
the % of job adverts with no skills overlap is: 0.425531914893617
the average # of lightcast skills we extract is: 10.872340425531915
the median # of lightcast skills we extract is: 10.0
the average # of lightcast skills lightcast extracts is: 6.74468085106383
the median # of lightcast skills lightcast extracts is: 5.0
of the job adverts with overlap, on average, 39.318289194020196 of lightcast skills are present in our current approach.
of the job adverts with overlap, the median is 33.33333333333333 of lightcast skills are present in our current approach.
of the job adverts with overlap, on average, 25.087275154612616 of our skills are present in lighcast skills.
of the job adverts with overlap, the median is 21.428571428571427 of our skills are present in lightcast skills.


### 3. ESCO occupations comparison analysis

In [9]:
print(f"{len(esco_occupations_df)} occupations (with 100 or more job adverts) in ESCO were also found in OJO.")
print(f"the average # of adverts per occupation (with 100 or more job adverts) is {esco_occupations_df.no_of_job_adverts.mean()}")
print(f"on average, {esco_occupations_df['skills_in_ojo_esco_percent'].mean()} percent of essential ESCO skills per occupation are were extracted from our algorithm.")
print(f"the median percent of essential ESCO skills per occupation are were extracted from our algorithm is {esco_occupations_df['skills_in_ojo_esco_percent'].median()}.")

print(f"the maximum % of skills mentioned in essential ESCO skills in OJO job adverts is {esco_occupations_df.iloc[0].skills_in_ojo_esco_percent}, for the occupation {esco_occupations_df.iloc[0].occupation}.")
print(f"there are {len(esco_occupations_df[esco_occupations_df['skills_in_ojo_esco_percent'] == 0.0])} occupations with no overlap.")

58 occupations (with 100 or more job adverts) in ESCO were also found in OJO.
the average # of adverts per occupation (with 100 or more job adverts) is 345.5344827586207
on average, 19.007832037090004 percent of essential ESCO skills per occupation are were extracted from our algorithm.
the median percent of essential ESCO skills per occupation are were extracted from our algorithm is 17.94871794871795.
the maximum % of skills mentioned in essential ESCO skills in OJO job adverts is 54.54545454545454, for the occupation project manager.
there are 1 occupations with no overlap.


In [10]:
random.seed(42)
occupation_name = random.choice(esco_occupations_df.occupation)

print(f'--top OJO skills for "{occupation_name}" that are not essential ESCO skills--')
print(list(esco_occupations_df[esco_occupations_df.occupation == occupation_name].in_ojo_not_esco))
print(f'--essential ESCO skills for "{occupation_name}" that were not extracted--')
print(list(esco_occupations_df[esco_occupations_df.occupation == occupation_name].in_esco_not_ojo))

--top OJO skills for "security officer" that are not essential ESCO skills--
[['respond to alarm systems', 'health not further defined', 'write English', 'interpret financial statements', 'be fluent in English', 'ordering vehicles', 'manage supply chain', 'complete administration', 'operating rail vehicles', 'give out licences', 'verify correctness of information', 'create design specifications', 'perform warehouse management activities', 'advising on legal, regulatory or procedural matters', 'improve quality of care', 'moving filled pallets', 'communicate efficiently and effectively', 'warehouse operations', 'moving or lifting materials, equipment, or supplies', 'working with machinery and specialised equipment', 'plan marketing campaign', 'complete balance sheet operations', 'strive to provide customers with the best possible service', 'ETL', 'verifying identities and documentation', 'fitting assistive devices', 'data warehouse', 'create current accounts', 'propose on risk management

In [11]:
random.seed(54)
occupation_name = random.choice(esco_occupations_df.occupation)

print(f'--top OJO skills for "{occupation_name}" that are not essential ESCO skills--')
print(list(esco_occupations_df[esco_occupations_df.occupation == occupation_name].in_ojo_not_esco))
print(f'--essential ESCO skills for "{occupation_name}" that were not extracted--')
print(list(esco_occupations_df[esco_occupations_df.occupation == occupation_name].in_esco_not_ojo))

--top OJO skills for "marketing manager" that are not essential ESCO skills--
[['respond to alarm systems', 'interaction', 'ordering vehicles', 'give out licences', 'technical drawings', 'provide information', 'obtain key information about projects', 'producing of different chilled prepared meals', 'warehouse operations', 'switching devices', 'improve personal skills', 'verifying identities and documentation', 'health', 'allocate resources according to community needs', 'making of surveying calculations', 'create current accounts', 'analyse your performance', "answer customers' inquiries", 'planning maintenance activities', 'delineate geographic sales areas', 'maintain payroll reports', 'install imaging equipment', 'organising booking operations', 'making decisions', 'e selling', 'keep operational standards high', 'nurture therapeutic relationships', 'directing operational activities', 'be reliable', 'maintain a clean work area', 'Iphone os', 'schedule appointments', 'providing informa

In [12]:
random.seed(72)
occupation_name = random.choice(esco_occupations_df.occupation)

print(f'--top OJO skills for "{occupation_name}" that are not essential ESCO skills--')
print(list(esco_occupations_df[esco_occupations_df.occupation == occupation_name].in_ojo_not_esco))
print(f'--essential ESCO skills for "{occupation_name}" that were not extracted--')
print(list(esco_occupations_df[esco_occupations_df.occupation == occupation_name].in_esco_not_ojo))

--top OJO skills for "software engineer" that are not essential ESCO skills--
[['respond to alarm systems', 'health not further defined', 'write English', 'be fluent in English', 'ordering vehicles', 'operating rail vehicles', 'give out licences', 'create design specifications', 'database development tools', 'obtain key information about projects', 'producing of different chilled prepared meals', 'improve quality of care', 'moving filled pallets', 'communicate efficiently and effectively', 'warehouse operations', 'data protection regulations', 'working with machinery and specialised equipment', 'negotiate terms with suppliers', 'plan marketing campaign', 'improve personal skills', 'ETL', 'verifying identities and documentation', 'health', 'making of surveying calculations', 'create current accounts', 'propose on risk management', 'budgeting principles', 'managing a website', 'utilise microsoft office', 'forms of pipeline', 'debt collection approaches', "answer customers' inquiries", 'd

#### 3.1 ESCO occupations comparison analysis - skill group level

In [99]:
esco_skills = esco_skills[~esco_skills['hierarchy_levels'].isna()]
esco_skills['hierarchy_levels'] = esco_skills['hierarchy_levels'].apply(ast.literal_eval)
esco_skills_dict = esco_skills.set_index('description')['hierarchy_levels'].to_dict()

In [119]:
def get_skill_level(skill_list):
    
    skill_group_list = [esco_skills_dict.get(i) for i in skill_list]
    
    skill_group_list_flat = []
    for skill_group in skill_group_list:
        if skill_group is not None:
            skill_group_flat = list(itertools.chain(*skill_group))
            for flat_group in skill_group_flat:
                if flat_group is not None and flat_group.count('.') == 2:
                    skill_group_list_flat.append(flat_group)
                
    return list(set(skill_group_list_flat))

In [120]:
#add skill groups to esco ojo comparisons 
skill_cols = ['in_both_ojo_esco', 'in_ojo_not_esco', 'in_esco_not_ojo']
for skill_col in skill_cols:
    esco_occupations_df[f'{skill_col}_skill_group'] = esco_occupations_df[skill_col].apply(get_skill_level)

In [132]:
#compare at the skill level group!

def compare_skill_groups(esco_skill_groups, ojo_skill_groups):
    
    esco_skill_groups = set(esco_skill_groups)
    ojo_skill_groups = set(ojo_skill_groups)
    
    in_both_ojo_esco = set.intersection(esco_skill_groups, ojo_skill_groups)
    skill_group_percent = (len(in_both_ojo_esco)/len(esco_skill_groups))* 100

    return skill_group_percent

In [133]:
esco_occupations_df['esco_skill_groups'] = esco_occupations_df['in_esco_not_ojo_skill_group'] + esco_occupations_df['in_both_ojo_esco_skill_group']
esco_occupations_df['esco_skill_groups'] = esco_occupations_df['esco_skill_groups'].apply(lambda x: list(set(x)))

esco_occupations_df['ojo_skill_groups'] = esco_occupations_df['in_ojo_not_esco_skill_group'] + esco_occupations_df['in_both_ojo_esco_skill_group']
esco_occupations_df['ojo_skill_groups'] = esco_occupations_df['ojo_skill_groups'].apply(lambda x: list(set(x)))

esco_occupations_df['skill_groups_in_ojo_esco_percent'] = esco_occupations_df.apply(lambda skill: compare_skill_groups(skill['esco_skill_groups'], skill['ojo_skill_groups']), axis=1)

In [150]:
print(f"on average, {esco_occupations_df.skill_groups_in_ojo_esco_percent.mean()} percent of essential ESCO skill groups per occupation are were extracted from our algorithm.")
print(f"the median percent of essential ESCO skills per occupation that were extracted from our algorithm is {esco_occupations_df['skill_groups_in_ojo_esco_percent'].median()}.")
print(f"{(len(esco_occupations_df[esco_occupations_df['skill_groups_in_ojo_esco_percent'] == 100])/len(esco_occupations_df))*100} % of occupations have 100% of ESCO essential skills at the skill group level.")
print(f"the occupation with the least amount of skill level group overlap is {esco_occupations_df.sort_values('skill_groups_in_ojo_esco_percent').iloc[0].occupation}, with {esco_occupations_df.sort_values('skill_groups_in_ojo_esco_percent').iloc[0].skill_groups_in_ojo_esco_percent}% overlap at the skill group level.")

on average, 95.27441974448372 percent of essential ESCO skill groups per occupation are were extracted from our algorithm.
the median percent of essential ESCO skills per occupation that were extracted from our algorithm is 97.63239002521715.
37.93103448275862 % of occupations have 100% of ESCO essential skills at the skill group level.
the occupation with the least amount of skill level group overlap is personal trainer, with 38.88888888888889% overlap at the skill group level.
